## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [19]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [20]:
print(resume)

 
Claudia  Fernandes  claudiacsf@gmail.com  |  LinkedIn |  GitHub Content,  Inbound  Growth,  Email  Automation,  SEO,  CRO  
Iʼm  a  SaaS  B2B  Content  Marketing  Manager  with  a  track  record  of  driving  brand  awareness  through  organic  growth  and  creating  content  that  resonates  with  technical  audiences .  Currently  working  to  stay  ahead  of  evolving  search  trends ,  and  leveraging  AI  to  improve  quality  while  accelerating  project  timelines.  
Present  一  06/2023  
Product  Marketing  Manager  一  Content  Marketing  Manager  
●  
Developed  and  executed  Assemblaʼs  content  strategy  from  the  ground  up,  scaling  organic  blog  traffic  from  20  to  2,500  monthly  visits  within  12  months .  This  high-intent  inbound  traffic  contributed  to  5%  of  Free  Trial  conversions  during  peak  months,  ultimately  driving  a  nearly  70%  uplift  in  subscription  rates .  The  Free  Trial-to-Paid  conversion  rate  surged  from  below  3%  to  1

In [16]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [17]:
name = "Claudia Fernandes"

In [23]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [22]:
system_prompt

"You are acting as Claudia Fernandes. You are answering questions on Claudia Fernandes's website, particularly questions related to Claudia Fernandes's career, background, skills and experience. Your responsibility is to represent Claudia Fernandes for interactions on the website as faithfully as possible. You are given a summary of Claudia Fernandes's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Claudia Fernandes. I'm a digital marketer based, an entrepreneur and wannabe AI engineer. I've watched all 16 seasons of Masterchef Australia.\n\n## Resume:\n \nClaudia  Fernandes  claudiacsf@gmail.com  |  LinkedIn |  GitHub Content,  Inbound  Growth,  Email  Automation,  SEO,  CRO  \nIʼm  a  SaaS  B2B  Content  Marketing  Manager  with  a  track  record  of  driving  brand  awareness  throug

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [24]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [26]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [27]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [39]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [40]:
import os
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

anthropic = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [50]:
def evaluate(reply, message, history) -> Evaluation:
    system_prompt = evaluator_system_prompt  # the actual string
    user_prompt = evaluator_user_prompt(reply, message, history)

    response = anthropic.messages.create(
        model="claude-3-5-sonnet-20240620",
        system=system_prompt,  # ✅ move system message here
        messages=[{"role": "user", "content": user_prompt}],
        max_tokens=1000
    )
    return response.content[0].text


In [57]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [52]:
reply

'No, I do not hold a patent. My expertise lies in digital marketing, content strategy, and leveraging AI for marketing initiatives. If you have any specific questions about my career or skills, feel free to ask!'

In [58]:
evaluate(reply, "do you hold a patent?", messages[:1])

'The response is acceptable. Here\'s my feedback:\n\n1. Directness: The agent clearly and directly answers the question with a "No," which is appropriate.\n\n2. Honesty: The response is honest about not holding a patent, which aligns with the information provided in Claudia\'s resume and background.\n\n3. Relevance: The agent provides context by mentioning Claudia\'s primary focus areas, which helps explain why she might not have a patent.\n\n4. Professionalism: The tone is professional and informative, maintaining the persona of Claudia Fernandes.\n\n5. Engagement: The agent invites further questions, which encourages continued interaction and shows openness to discussing other aspects of Claudia\'s experience.\n\n6. Accuracy: The response aligns with the information provided in Claudia\'s resume, which doesn\'t mention any patents.\n\nOverall, this is a well-crafted, appropriate response that accurately represents Claudia Fernandes based on the given information.'

In [54]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [59]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [60]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/claudia/course/agents/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/claudia/course/agents/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/claudia/course/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/claudia/course/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/claudia/course/agents/.venv/lib/python3.12/site-packages/gradio/utils.py", line 861, in async_wrapper